Domain – Media

focus – optimize selection process


Business challenge/requirement

Motion Studios is the largest Radio production house in Europe. Their total revenue $ 1B+. Company has launched a new reality show – "The Star RJ". The show is about finding a new Radio Jockey who will be the star presenter on upcoming shows.

In first round participants have to upload their voice clip online and the clip will be evaluated by experts for selection into the next round. There is a separate team in the first round for evaluation of male and female voice.


Response to the show is unprecedented and company is flooded with voice clips.

You as a ML expert have to classify the voice as either male/female so that first level of filtration is quicker.

Key issues
Voice sample are across accents



Data volume

-	Approx 3000 records – file voice-classification.csv Fields in Data

•	meanfreq: mean frequency (in kHz)

•	sd: standard deviation of frequency

•	median: median frequency (in kHz)

•	Q25: first quantile (in kHz)

•	Q75: third quantile (in kHz)

•	IQR: interquantile range (in kHz)

•	skew: skewness (see note in specprop description)

•	kurt: kurtosis (see note in specprop description)

•	sp.ent: spectral entropy

•	sfm: spectral flatness

•	mode: mode frequency
•	centroid: frequency centroid (see specprop)

•	peakf: peak frequency (frequency with highest energy)

•	meanfun: average of fundamental frequency measured across acoustic signal

•	minfun: minimum fundamental frequency measured across acoustic signal

•	maxfun: maximum fundamental frequency measured across acoustic signal

•	meandom: average of dominant frequency measured across acoustic signal

•	mindom: minimum of dominant frequency measured across acoustic signal

•	maxdom: maximum of dominant frequency measured across acoustic signal

•	dfrange: range of dominant frequency measured across acoustic signal

•	modindx: modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range

•	label: male or female




Additional information
- NA



Business benefits

Since "The Star RJ" is a reality show, time to select candidates is very short. The whole success of the show and hence the profits depends upon quick and smooth execution

Note: Use SVM , Naïve Bayce  and GridSearchCV to classify the data 


In [4]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

In [6]:
os.chdir('D:\\top mentor assignment\\Batch 98 Day 57\\ML Case Studies with Answer\\CS 33 Ans -Supervised Learnig-4\\')

In [7]:
os.getcwd()

'D:\\top mentor assignment\\Batch 98 Day 57\\ML Case Studies with Answer\\CS 33 Ans -Supervised Learnig-4'

In [8]:
df = pd.read_csv('voice-classification.csv')
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   3168 non-null   float64
 20  label     3168 non-null   obje

In [10]:
df.describe()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
count,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000
mean,0.180907,0.057126,0.185621,0.140456,0.224765,0.084309,3.140168,36.568461,0.895127,0.408216,0.165282,0.180907,0.142807,0.036802,0.258842,0.829211,0.052647,5.047277,4.994630,0.173752
std,0.029918,0.016652,0.036360,0.048680,0.023639,0.042783,4.240529,134.928661,0.044980,0.177521,0.077203,0.029918,0.032304,0.019220,0.030077,0.525205,0.063299,3.521157,3.520039,0.119454
min,0.039363,0.018363,0.010975,0.000229,0.042946,0.014558,0.141735,2.068455,0.738651,0.036876,0.000000,0.039363,0.055565,0.009775,0.103093,0.007812,0.004883,0.007812,0.000000,0.000000
25%,0.163662,0.041954,0.169593,0.111087,0.208747,0.042560,1.649569,5.669547,0.861811,0.258041,0.118016,0.163662,0.116998,0.018223,0.253968,0.419828,0.007812,2.070312,2.044922,0.099766
50%,0.184838,0.059155,0.190032,0.140286,0.225684,0.094280,2.197101,8.318463,0.901767,0.396335,0.186599,0.184838,0.140519,0.046110,0.271186,0.765795,0.023438,4.992188,4.945312,0.139357
75%,0.199146,0.067020,0.210618,0.175939,0.243660,0.114175,2.931694,13.648905,0.928713,0.533676,0.221104,0.199146,0.169581,0.047904,0.277457,1.177166,0.070312,7.007812,6.992188,0.209183
max,0.251124,0.115273,0.261224,0.247347,0.273469,0.252225,34.725453,1309.612887,0.981997,0.842936,0.280000,0.251124,0.237636,0.204082,0.279114,2.957682,0.458984,21.867188,21.843750,0.932374


In [12]:
df.isnull().sum()


meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64

In [13]:
print ( "Shape of Data:" , df.shape)

print("Total number of labels: {}".format(df.shape[0]))

print("Number of male: {}".format(df[df.label == 'male'].shape[0]))

print("Number of female: {}".format(df[df.label == 'female'].shape[0]))

Shape of Data: (3168, 21)
Total number of labels: 3168
Number of male: 1584
Number of female: 1584


In [14]:
X=df.iloc[:, :-1]
print (df.shape)
print (X.shape)

(3168, 21)
(3168, 20)


In [16]:
y=df.iloc[:,-1]
print (y.shape)

(3168,)


In [17]:
gender_encoder = LabelEncoder()
y = gender_encoder.fit_transform(y)
print (y)

[1 1 1 ... 0 0 0]


In [18]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [21]:
svc_model=SVC()
svc_model.fit(X_train,y_train)
y_pred=svc_model.predict(X_test)

In [22]:
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.9737118822292324


In [23]:
print(confusion_matrix(y_test,y_pred))

[[458  13]
 [ 12 468]]


In [24]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       471
           1       0.97      0.97      0.97       480

    accuracy                           0.97       951
   macro avg       0.97      0.97      0.97       951
weighted avg       0.97      0.97      0.97       951



In [25]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]}

grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.0s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.0s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.0s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.0s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.0s
[CV] END ..................................C=0.1, gamma=0.01; total time=   0.0s
[CV] END ..................................C=0.1

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001]},
             verbose=2)

In [26]:
best_params = grid.best_params_
print (best_params)


{'C': 1, 'gamma': 0.1}


In [27]:
grid_predictions = grid.predict(X_test)

In [28]:
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,grid_predictions))

Accuracy Score:
0.9747634069400631


In [29]:
print(confusion_matrix(y_test,grid_predictions))

[[459  12]
 [ 12 468]]


In [30]:
print(classification_report(y_test,grid_predictions))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       471
           1       0.97      0.97      0.97       480

    accuracy                           0.97       951
   macro avg       0.97      0.97      0.97       951
weighted avg       0.97      0.97      0.97       951



In [31]:
GNB= GaussianNB()
GNB.fit(X_train,y_train)
GNB_pred = GNB.predict(X_test)

In [32]:
print (GNB_pred)
print (pd.DataFrame(GNB_pred).groupby(0).agg({0:np.size}))
print('\n Accuracy Score')
print (metrics.accuracy_score (y_test,GNB_pred))
print('\nClassification Report')
print(classification_report (y_test,GNB_pred))
print('Confusion Matrix')
print (confusion_matrix (y_test,GNB_pred))

[1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 1 1 0 0 0 0 1 0
 0 1 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 1 1
 1 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0
 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 1
 0 1 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 1 1
 0 0 1 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0
 1 1 1 0 1 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0
 1 1 0 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 1 1 1 1
 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 1 0 1 0
 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 1 1 1 1
 0 1 1 1 0 0 1 0 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 1 0 1 0 1
 1 1 1 0 1 1 0 0 0 0 1 0 

In [33]:
param_grid = {} 
grid = GridSearchCV(GaussianNB(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


GridSearchCV(estimator=GaussianNB(), param_grid={}, verbose=2)

In [34]:
best_params = grid.best_params_
print (best_params)

{}


In [35]:
grid_predictions = grid.predict(X_test)

In [36]:
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,grid_predictions))

Accuracy Score:
0.8696109358569927


In [37]:
print(confusion_matrix(y_test,grid_predictions))

[[406  65]
 [ 59 421]]


In [38]:
print(classification_report(y_test,grid_predictions))

              precision    recall  f1-score   support

           0       0.87      0.86      0.87       471
           1       0.87      0.88      0.87       480

    accuracy                           0.87       951
   macro avg       0.87      0.87      0.87       951
weighted avg       0.87      0.87      0.87       951

